In [ ]:
!python -m pip install cn_clip

In [ ]:
import os
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datetime import datetime, timedelta
def get_avg_time(time_1, time_2):
    # 将时间字符串转换为时间间隔
    format_string = '%H:%M:%S.%f'
    time1_obj = datetime.strptime(time_1, format_string)
    time2_obj = datetime.strptime(time_2, format_string)

    # 计算时间间隔的总和
    total_time = time1_obj - datetime.min + (time2_obj - datetime.min)

    # 计算平均时间间隔
    average_time = total_time / 2

    # 将平均时间间隔格式化为相同的时间字符串格式
    average_time_str = (datetime.min + average_time).time().strftime(format_string)
    return average_time_str

In [ ]:
def getTimeAndText(ass_path):
    with open(ass_path, 'r', encoding='utf-16') as file:
        lines = file.readlines()
    keyword = "zhengwen"
    avg_time_list = []
    text_list = []
    for line in lines:
        if line.startswith('Dialogue') and keyword in line:
            content = line.split(":", 1)[1].strip()
            line_list = content.split(",")
            avg_time = get_avg_time(line_list[1], line_list[2])
            avg_time_list.append(avg_time)
            text_list.append(line_list[-1])
    return avg_time_list, text_list

In [ ]:
import torch
from PIL import Image

import cn_clip.clip as clip
from cn_clip.clip import tokenize, load_from_name

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, processer = load_from_name("ViT-H-14", device=device)

100%|█████████████████████████████████████| 3.57G/3.57G [01:42<00:00, 37.5MiB/s]


Loading vision model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/ViT-H-14.json
Loading text model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/RoBERTa-wwm-ext-large-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': 32, 'vision_width': 1280, 'vision_head_width': 80, 'vision_patch_size': 14, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 1024, 'text_initializer_range': 0.02, 'text_intermediate_size': 4096, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 16, 'text_num_hidden_layers': 24, 'text_type_vocab_size': 2}


In [ ]:
model.eval()

CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)


In [ ]:
import json
import subprocess

def extract_frame_at_time(video_path, output_path, time_stamp):
    ffmpeg_cmd = [
        'ffmpeg',
        '-ss', time_stamp,
        '-i', video_path,
        '-vframes', '1',
        f"\'{output_path}\'"
    ]

    subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# for id in [str(x).zfill(2) for x in range(15, 29)]:

for k in range(1,15):
    save_path = f"/content/drive/MyDrive/GPTData/Haruhi_data/{k:0=2d}/"
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    data = []
    ass_path = f"[CASO][Suzumiya_Haruhi_no_Yuuutsu][{k:0=2d}]"
    video_path = f"[CASO][Suzumiya_Haruhi_no_Yuuutsu][{k:0=2d}]"

    directory = "/content/drive/MyDrive/GPTData/Haruhi_all"
    for filename in os.listdir(directory):
        if filename.endswith("ass") and filename.startswith(ass_path):
            ass_path = filename
        if filename.endswith("mkv") and filename.startswith(video_path):
            video_path = filename

    ass_path = os.path.join(directory, ass_path)
    video_path = os.path.join(directory, video_path)

    avg_time_list, text_list = getTimeAndText(ass_path)

    for i in range(len(avg_time_list)):
        time_stamp = avg_time_list[i]
        output_path = save_path + f'{text_list[i]}{avg_time_list[i]}.jpg'
        extract_frame_at_time(video_path, output_path, time_stamp)
        data.append({"text":str(text_list[i]), "timestamp":str(avg_time_list[i]), "path":f"{text_list[i]}{avg_time_list[i]}.jpg"})

    with open(f'{save_path}/info_{k:0=2d}.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
for k in range(1,15):
    save_path = f"/content/drive/MyDrive/GPTData/Haruhi_data/{k:0=2d}/"
    base_dir = save_path

    info_dir = os.path.join(save_path, f"info_{k:0=2d}.json")

    data = []

    with open(info_dir, 'r', encoding='utf-8') as f:
        info = json.load(f)

    for item in info:
        image = processer(Image.open(f"{base_dir}/{item['path']}")).unsqueeze(0).to(device)
        text  = tokenize([item['text']]).to(device)

        with torch.no_grad():
            image_embedding = model.encode_image(image)
            text_embedding  = model.encode_text(text)

            image_embedding /= image_embedding.norm(dim=-1, keepdim=True)
            text_embedding  /= text_embedding.norm(dim=-1, keepdim=True)

        item['text_embedding'] = text_embedding[0].tolist()
        item['image_embedding'] = image_embedding[0].tolist()

        data.append(item)

    with open(os.path.join(save_path, f"info_{k:0=2d}_embedding.json"), "w", encoding="utf-8") as f:
        json.dump(data,f,ensure_ascii=False,indent=4)